In [80]:
import pandas as pd
import json
import numpy as np
import jenks

In [81]:
momc = pd.read_csv('../scratch/momc_meta.csv').set_index('KOD_MOaMC').to_dict(orient='index')
ob = pd.read_csv('../scratch/obce_meta.csv').set_index('KOD_OBEC').to_dict(orient='index')

In [82]:
with open('../scratch/okrsky.json', encoding='utf-8') as f:
    jsn = json.load(f)

In [83]:
meta = {}

In [84]:
# zpracovani MC
for mc in momc:
    meta[mc] = {
        'mc': momc[mc]['NAZ_MOaMC'], 
        'obec': momc[mc]['NAZ_OBEC'], 
        'okres': momc[mc]['NAZ_LAU1'], 
        'kraj': momc[mc]['NAZ_CZNUTS3']
    }

In [85]:
# zpracovani obci
for o in ob:
    meta[o] = {
        'mc': None, 
        'obec': ob[o]['NAZ_OBEC'], 
        'okres': ob[o]['NAZ_LAU1'], 
        'kraj': ob[o]['NAZ_CZNUTS3']
    }

In [86]:
for fe in jsn['features']:
    momc_kod = fe['properties']['MomcKod']
    obec_kod = fe['properties']['ObecKod']
    if momc_kod != None:
        kd = momc_kod
    else:
        kd = obec_kod
    
    fe['properties'].update(meta[kd])

In [87]:
okrs = pd.read_csv('../scratch/pet1.csv', sep=';')

In [88]:
okrs['okid'] = okrs.OBEC.astype('str') + '_' + okrs.OKRSEK.astype('str')

In [89]:
cls = [
    'CHYBA',
    'KC_1',
    'KC_2',
    'KC_3',
    'KC_4',
    'HLASY_10',
    'HLASY_11',
    'HLASY_12',
    'HLASY_13',
    'HLASY_14',
    'HLASY_15',
    'HLASY_16',
    'HLASY_17',
    'HLASY_18',
    'HLASY_19',
    'HLASY_20',
    'KC_SUM',
    'KOLO',
    'OBEC',
    'OKRES',
    'OKRSEK',
    'OPRAVA',
    'POSL_KAND',
    'TYP_FORM'
]
okrs.drop(cls, inplace=True, axis=1)

In [90]:
okrs = okrs.set_index('okid').to_dict(orient='index')

In [91]:
for fe in jsn['features']:
    momc_kod = fe['properties']['MomcKod']
    obec_kod = fe['properties']['ObecKod']
    if momc_kod != None:
        kd = str(momc_kod) + '_' + str(fe['properties']['Cislo'])
    else:
        kd = str(obec_kod) + '_' + str(fe['properties']['Cislo'])
    
    try:
        fe['properties'].update(okrs[kd])
    except:
        jsn['features'].pop(jsn['features'].index(fe)) #možná časem udělat hezky, ne je jen vyhodit

In [ ]:
with open('../scratch/map.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(jsn))

In [ ]:
#breaks
k = 5
nbAttempt = 4

target = open('../js/breaks.js', 'w')
target.truncate()
target.write('var breaks = {\n')

for i in range(1, 10):
    vals = []
    for ftr in jsn['features']:
        vals.append((ftr['properties']['HLASY_' + str(i).rjust(2, '0')] * 1.0) / ftr['properties']['PL_HL_CELK'])
        
    vals = np.trim_zeros(vals)
    if np.mean(vals) == 0:
        continue
    else:
        brks = [0, 0, 0, 0]
        try:
            casp = jenks.jenksCaspall(vals, k, nbAttempt, initStyle='kpp')
            brks = casp.breaks
        except:
            pass
        target.write('HLASY_' + str(i).rjust(2, '0') + ': ' + str(brks) + ',\n')
        
target.write("};")
target.close()